In [1]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import joblib
import wave
import hashlib
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from scipy.io import wavfile
from scipy.fft import fft, ifft
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# === STEP 1: Load Audio Data (Scene Fake Dataset) ===
def load_audio_features(dataset_path):
    features, labels = [], []
    for label in ["real", "fake"]:
        folder = os.path.join(dataset_path, label)
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            y, sr = librosa.load(file_path, sr=16000)  # Load audio
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            features.append(np.mean(mfcc, axis=1))
            labels.append(0 if label == "real" else 1)  # 0 = real, 1 = fake
    return np.array(features), np.array(labels)

# === STEP 2: Watermarking using Spread Spectrum ===
def spread_spectrum_watermark(audio_file, watermark_bits, output_file, alpha=0.01):
    rate, audio = wavfile.read(audio_file)
    audio = audio.astype(np.float32)
    
    # Generate pseudo-random sequence
    pseudo_noise = np.random.randn(len(audio))
    watermark_signal = pseudo_noise * alpha * watermark_bits.mean()

    # Embed watermark
    audio += watermark_signal
    audio = np.int16(audio / np.max(np.abs(audio)) * 32767)
    wavfile.write(output_file, rate, audio)
    print("Watermark embedded.")

# === STEP 3: Digital Signature (RSA) ===
# Generate RSA Keys
def generate_rsa_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    
    # Save keys
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))
    
    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))
    return private_key, public_key

# Sign a File
def sign_audio(file_path, private_key):
    with open(file_path, "rb") as f:
        file_data = f.read()
        hash_value = hashlib.sha256(file_data).digest()
        signature = private_key.sign(
            hash_value,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
    with open(f"{file_path}.sig", "wb") as f:
        f.write(signature)
    print("Digital Signature Created.")

# Verify Signature
def verify_audio(file_path, public_key):
    with open(file_path, "rb") as f:
        file_data = f.read()
        hash_value = hashlib.sha256(file_data).digest()

    with open(f"{file_path}.sig", "rb") as f:
        signature = f.read()

    try:
        public_key.verify(
            signature,
            hash_value,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        print("Signature Verified: Authentic Audio.")
        return True
    except:
        print("Signature Verification Failed!")
        return False

# === STEP 4: Machine Learning Classification (Detect Fake Audio) ===
def train_model(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    print(f"Model Accuracy: {model.score(X_test, y_test) * 100:.2f}%")
    joblib.dump(model, "audio_classifier.pkl")

def classify_audio(file_path, model):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    feature = np.mean(mfcc, axis=1).reshape(1, -1)
    prediction = model.predict(feature)
    return "Real" if prediction == 0 else "Fake"

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import joblib
import wave
import hashlib
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from scipy.io import wavfile
from scipy.fft import fft, ifft
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# === STEP 1: Load Audio Data (Scene Fake Dataset) ===
def load_audio_features(dataset_path):
    features, labels = [], []
    for label in ["real", "fake"]:
        folder = os.path.join(dataset_path, label)
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            y, sr = librosa.load(file_path, sr=16000)  # Load audio
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            features.append(np.mean(mfcc, axis=1))
            labels.append(0 if label == "real" else 1)  # 0 = real, 1 = fake
    return np.array(features), np.array(labels)

# === STEP 2: Watermarking using Spread Spectrum ===
def spread_spectrum_watermark(audio_file, watermark_bits, output_file, alpha=0.01):
    rate, audio = wavfile.read(audio_file)
    audio = audio.astype(np.float32)
    
    # Generate pseudo-random sequence
    pseudo_noise = np.random.randn(len(audio))
    watermark_signal = pseudo_noise * alpha * watermark_bits.mean()

    # Embed watermark
    audio += watermark_signal
    audio = np.int16(audio / np.max(np.abs(audio)) * 32767)
    wavfile.write(output_file, rate, audio)
    print(f"Watermark embedded in {output_file}")

# === STEP 3: Digital Signature (RSA) ===
# Generate RSA Keys
def generate_rsa_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    
    # Save keys
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))
    
    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))
    return private_key, public_key

# Sign a File
def sign_audio(file_path, private_key):
    with open(file_path, "rb") as f:
        file_data = f.read()
        hash_value = hashlib.sha256(file_data).digest()
        signature = private_key.sign(
            hash_value,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
    with open(f"{file_path}.sig", "wb") as f:
        f.write(signature)
    print(f"Digital Signature Created for {file_path}")

# Verify Signature
def verify_audio(file_path, public_key):
    with open(file_path, "rb") as f:
        file_data = f.read()
        hash_value = hashlib.sha256(file_data).digest()

    try:
        with open(f"{file_path}.sig", "rb") as f:
            signature = f.read()
        public_key.verify(
            signature,
            hash_value,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        print("Signature Verified: Authentic Audio.")
        return True
    except:
        print("Signature Verification Failed!")
        return False

# === STEP 4: Machine Learning Classification (Detect Fake Audio) ===
def train_model(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    print(f"Model Accuracy: {model.score(X_test, y_test) * 100:.2f}%")
    joblib.dump(model, "audio_classifier.pkl")

def classify_audio(file_path, model):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    feature = np.mean(mfcc, axis=1).reshape(1, -1)
    prediction = model.predict(feature)
    return "Real" if prediction == 0 else "Fake"

# === MAIN FUNCTION TO EXECUTE WORKFLOW ===
def main():
    dataset_path = "./archive/"  # Folder containing "real" and "fake" subfolders
    
    # 1. Load Dataset & Train Model
    print("Loading Dataset and Extracting Features...")
    features, labels = load_audio_features(dataset_path)
    print("Training Model...")
    train_model(features, labels)
    
    # 2. Generate RSA Keys for Signing
    print("Generating RSA Keys...")
    private_key, public_key = generate_rsa_keys()
    
    # 3. Apply Watermarking & Digital Signing
    test_audio = "./"
    watermarked_audio = "test_audio_watermarked.wav"
    print("Embedding Watermark...")
    spread_spectrum_watermark(test_audio, np.random.randint(0, 2, size=100), watermarked_audio)
    
    print("Signing Audio File...")
    sign_audio(watermarked_audio, private_key)
    
    # 4. Testing Phase
    print("Testing with New Audio File...")
    
    # Step 1: Verify Watermark
    print("Checking for Watermark...")
    # Assume watermark detection function exists; skipping implementation
    watermark_detected = True  # Simulated detection
    
    if not watermark_detected:
        print("Watermark Missing! Classified as SPOOFED.")
        return
    
    # Step 2: Verify Digital Signature
    print("Verifying Digital Signature...")
    if not verify_audio(watermarked_audio, public_key):
        print("Invalid Digital Signature! Classified as SPOOFED.")
        return
    
    # Step 3: Extract Features & Classify
    print("Extracting Features and Classifying Audio...")
    model = joblib.load("audio_classifier.pkl")
    result = classify_audio(watermarked_audio, model)
    
    print(f"Final Decision: {result}")

if __name__ == "__main__":
    main()


In [2]:
import numpy as np
import librosa
import librosa.feature
import hashlib
import random
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
import soundfile as sf
import uuid

# --- Audio Watermarking (AudioSeal) ---
def embed_watermark(audio, watermark_strength=0.1):
    """Embeds a simple watermark into the audio."""
    watermark = np.random.rand(len(audio)) * watermark_strength
    watermarked_audio = audio + watermark
    return watermarked_audio, watermark

def extract_watermark(watermarked_audio, original_audio=None):
    """Extracts the watermark. If original is not given, extracts noise as watermark"""
    if original_audio is None:
        return watermarked_audio #returning the audio itself as a fail safe for water mark fail.
    return watermarked_audio - original_audio

def verify_watermark(extracted_watermark, original_watermark, threshold=0.05):
    """Verifies the extracted watermark against the original."""
    if original_watermark is None:
        return False #if original watermark is missing, fail.
    difference = np.mean(np.abs(extracted_watermark - original_watermark))
    return difference < threshold

# --- Digital Signature (Cryptographic Hashing) ---
def generate_signature(audio_path):
    """Generates a digital signature for an audio file."""
    with open(audio_path, 'rb') as f:
        audio_bytes = f.read()
        signature = hashlib.sha256(audio_bytes).hexdigest()
    return signature

def verify_signature(audio_path, original_signature):
    """Verifies the digital signature of an audio file."""
    return generate_signature(audio_path) == original_signature

# --- Feature Extraction (IGTCC, GTCC) ---
def extract_features(audio, sr):
    """Extracts IGTCC and GTCC features."""
    igtcc = librosa.feature.chroma_cens(y=audio, sr=sr)
    gtcc = librosa.feature.gtcc(y=audio, sr=sr)
    return np.concatenate((igtcc.flatten(), gtcc.flatten()))

# --- Machine Learning Model (ResNet50 / MLP) ---
def build_resnet50_model(input_shape, num_classes):
    """Builds a ResNet50 model."""
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_mlp_model(input_shape, num_classes):
    """Builds an MLP model."""
    model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=500)
    return model

def load_scene_fake_data(directory):
    """Loads SceneFake data from a given directory."""
    features = []
    labels = []
    audio_paths = []
    for label_dir in ['real', 'fake']:
        label = 0 if label_dir == 'real' else 1
        for filename in os.listdir(os.path.join(directory, label_dir)):
            if filename.endswith('.wav'):
                audio_path = os.path.join(directory, label_dir, filename)
                audio, sr = librosa.load(audio_path, sr=None)
                features.append(extract_features(audio, sr))
                labels.append(label)
                audio_paths.append(audio_path)

    return np.array(features), np.array(labels), audio_paths

def train_model_scene_fake(train_dir, model_type='mlp'):
    """Trains the model with SceneFake data, watermarking and signature."""
    features, labels, audio_paths = load_scene_fake_data(train_dir)
    watermark_db = {}
    signature_db = {}
    watermarked_paths = []

    for i in range(len(audio_paths)):
        if labels[i] == 0:  # Only watermark real audio
            audio, sr = librosa.load(audio_paths[i], sr=None)
            watermarked_audio, original_watermark = embed_watermark(audio)
            signature = generate_signature(audio_paths[i])
            temp_watermarked_path = f"temp_watermarked_{uuid.uuid4()}.wav"
            sf.write(temp_watermarked_path, watermarked_audio, sr)
            features[i] = extract_features(watermarked_audio, sr) #overwrites the real features with watermarked features.
            watermark_db[os.path.basename(audio_paths[i])] = original_watermark
            signature_db[os.path.basename(audio_paths[i])] = signature
            watermarked_paths.append(temp_watermarked_path)
    if model_type == 'resnet':
        features = features.reshape(features.shape[0], 12, 12, 1) #Reshape for resnet.
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
        y_train_categorical = to_categorical(y_train)
        y_test_categorical = to_categorical(y_test)
        model = build_resnet50_model(input_shape=(12, 12, 1), num_classes=2)
        model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_test, y_test_categorical))
        return model, watermark_db, signature_db

    elif model_type == 'mlp':
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
        model = build_mlp_model(input_shape=features.shape[1], num_classes=2)
        model.fit(X_train, y_train)
        return model, watermark_db, signature_db

def test_model_scene_fake(audio_path, model, watermark_db, signature_db, model_type='mlp'):
    """Tests the model with SceneFake data, watermark and signature verification."""
    filename = os.path.basename(audio_path)
    audio, sr = librosa.load(audio_path, sr=None)

    # Watermark Verification
    if filename in watermark_db:
        original_audio, sr_original = librosa.load(os.path.join(train_dir, 'real', filename), sr=None)
        extracted_watermark = extract_watermark(audio, original_audio)
        if not verify_watermark(extracted_watermark, watermark_db[filename]):
            return "SPOOFED (Watermark Failed)"
    else:
        extracted_watermark = extract_watermark(audio)
        if np.mean(np.abs(extracted_watermark)) <0.0001:
            return "SPOOFED (Watermark Missing)"

    # Signature Verification
    if filename in signature_db:
        if not verify_signature(audio_path, signature_db[filename]):
            return "SPOOFED (Signature Failed)"

    # Feature Extraction and ML Classification
    features = extract_features(audio, sr)
    features = features.reshape(1, -1)

    if model_type == 'resnet':
        features = features.reshape(1, 12, 12, 1)
        prediction = model.predict(features)
        predicted_class = np.argmax(prediction)
    elif model_type == 'mlp':
        predicted_class = model.predict(features)[0]

    return "REAL" if predicted_class == 0 else "SPOOFED (ML)"

# --- Example Usage ---
train_dir = './archive/dev/'  # Path to the 'dev' directory
eval_dir = './archive/eval/' # path to the eval directory
# test_dir = './archive' # path to the test directory

model, watermark_db, signature_db = train_model_scene_fake(train_dir, model_type='mlp') # Train on dev set.

# Example Test on the Eval set:
for label_dir in ['real','fake']:
    for filename in os.listdir(os.path.join(eval_dir,label_dir)):
        if filename.endswith(".wav"):
            audio_path = os.path.join(eval_dir, label_dir, filename)
            result = test_model_scene_fake(audio_path, model, watermark_db, signature_db)
            print(f"{audio_path}: {result}")


AttributeError: No librosa.feature attribute gtcc